In [3]:
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_auth = get_key('koray_data')

my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

file_url = '/search/?type=FileFastq&status=uploading&status=upload failed&limit=all'
#file_url = '/search/?type=FileReference&status=uploading&status=upload failed&limit=all'
#file_url = '/search/?type=FileProcessed&status=uploading&status=upload failed&limit=all'
file_url = "/search/?lab.display_title=Job+Dekker%2C+UMMS&status=uploading&status=upload+failed&type=FileFastq"


#to do 
delete old
release new

https://data.4dnucleome.org/search/?content_md5sum=No+value&status=released&type=FileFastq
    

file_url = '/search/?md5sum=No+value&status=released&status=released+to+project&status=uploaded&type=FileCalibration&type=FileFastq&type=FileReference&type=FileMicroscopy'

all_files = ff_utils.search_metadata(file_url, key=my_auth)

print(len(all_files))

65


In [8]:
print([i['@id'] for i in all_files])

['/files-reference/4DNFIS1ZVUWO/', '/files-reference/4DNFIBEEN92C/', '/files-fastq/4DNFIPLDA6QM/', '/files-fastq/4DNFI1W4JRH3/', '/files-fastq/4DNFIXS2US9H/', '/files-fastq/4DNFII9FCYYW/', '/files-fastq/4DNFIUAPAXL1/', '/files-fastq/4DNFISH4FPSW/', '/files-fastq/4DNFIUX841K8/', '/files-fastq/4DNFIS94W8RK/', '/files-fastq/4DNFIJEUX4GL/', '/files-microscopy/4DNFI4KZIPV7/', '/files-microscopy/4DNFIW3788O9/', '/files-reference/4DNFI0NK4G14/', '/files-reference/4DNFI3HVC1SE/', '/files-reference/4DNFI3HVU20D/', '/files-reference/4DNFIMIV7AHJ/', '/files-reference/4DNFI9P2SCZ7/', '/files-reference/4DNFI4FLO6VF/', '/files-fastq/4DNFIYU8DDMX/', '/files-reference/4DNFI3UBJ3HZ/', '/files-fastq/4DNFIT2OY9HJ/', '/files-fastq/4DNFIVJZKL9R/', '/files-fastq/4DNFISY5ED3Q/', '/files-fastq/4DNFIDCLJ6LY/', '/files-fastq/4DNFI82SFXBP/', '/files-fastq/4DNFIF58NXJY/', '/files-fastq/4DNFIPBQPYG5/', '/files-fastq/4DNFIZUZHKUP/', '/files-fastq/4DNFIWE1ZP5N/', '/files-fastq/4DNFINU3572Y/', '/files-fastq/4DNFIJ3SS

In [4]:
run_md5 = input("Do you wanna run md5 if missing? (md5/none)")

from contextlib import contextmanager
import sys, os

# supress output when activated (with supress_stdout():)
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout


no_md5 = 0
no_s3_file = 0
started_missing = 0
started_weird = 0

printn = 0
counter = 0
for a_file in all_files:

    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            print("Deleted File", a_file)
            continue
    except:
        print(a_file)
        break
        
    if counter-printn > 100:
        rep = "---{c} of {m} missing md5, {s3} had no files on s3".format(c=counter, m=no_md5, s3=no_s3_file)
        rep2 = "---{c} of {s} started, {sw} started (had complete run before)".format(c=counter, s=started_missing, sw=started_weird)
        print(rep)
        print(rep2)
        printn = counter

    if 'FileProcessed' in a_file['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket       
 
    # check if file is in s3
    with suppress_stdout():
        head_info = my_s3_util.does_key_exist(a_file['upload_key'], my_bucket)
    
    if not head_info:
        no_s3_file += 1
        #print(a_file['accession'], a_file.get('aliases')[0], "does not have a file in S3")
        continue
        
    # print(head_info['LastModified'])

    file_id = a_file['accession']
    status = a_file.get('status')
    attributions = get_attribution(a_file)
    
    # is there md5sum for gzip
    if not a_file.get('md5sum'):
        print(file_id,"does not have the md5sum calculated during upload")
        no_md5 += 1
        
        inp_f = {'input_file':a_file['@id']}
        run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
   
    
#     # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
#     md5_report = get_wfr_out(a_file['uuid'], "md5", my_auth, md_qc = True, run=12)
    
#     if md5_report['status'] == 'running':
#         print(file_id, 'still running')
#         continue 
    
#     # Most probably the trigger did not work, and we run it manually 
#     if status in ["uploading", "upload failed"] and md5_report['status'] != 'complete':
#         # if not, shall we run it?
#         if run_md5 == 'md5':
            
#             print('md5 running for', file_id)
#             inp_f = {'input_file':a_file['@id']}
#             run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
#             started_missing += 1
#             print()
#         # user does not want it to be run, so just report
#         else:
#             print('md5 run missing for', file_id)
    
#     # There is a successful run, but status is not switched, happens whe a file is reuploaded. 
#     elif status in ["uploading", "upload failed"] and md5_report['status'] == 'complete':
#         # if not, shall we run it?
#         if run_md5 == 'md5':
#             print('md5 running for', file_id, 'had a successful run and wrong status')
#             inp_f = {'input_file':a_file['@id']}
#             run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
#             started_weird += 1
#             print()
#         else:
#             print('md5 run missing for', file_id, 'had a successful run and wrong status')
#     else:
#         print("SHOULD NOT PASS", a_file['accession'], md5_report)
        
#     elif md5_status != 'complete':
#         if a_file.get('content_md5sum'):
#             print('md5run not linked to file')
#             if run_md5 == 'md5':
#                 print('md5 running for', file_id)
#                 inp_f = {'input_file':a_file['@id']}
#                 run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
#                 print()
#         else:
#             print(file_id,md5_status, status, 'switched status file does not have md5 run or content_md5sum')
#             # if not, shall we run it?
#             if run_md5 == 'md5':
#                 print('md5 running for', file_id)
#                 inp_f = {'input_file':a_file['@id']}
#                 run_missing_wfr(step_settings('md5', 'no_organism', attributions), inp_f, file_id, my_auth, my_env, tag="")
#                 print()
#     print()

rep = "FINAL : {c} of {m} missing md5, {s3} had no files on s3".format(c=counter, m=no_md5, s3=no_s3_file)
print(rep)
rep2 = "FINAL : {c} of {s} started, {sw} started (had complete run before)".format(c=counter, s=started_missing, sw=started_weird)
print(rep2)

Do you wanna run md5 if missing? (md5/none)md5
4DNFIS1ZVUWO does not have the md5sum calculated during upload


4DNFIBEEN92C does not have the md5sum calculated during upload


4DNFIPLDA6QM does not have the md5sum calculated during upload


4DNFI1W4JRH3 does not have the md5sum calculated during upload


4DNFIXS2US9H does not have the md5sum calculated during upload


...

4DNFIIDYXJY6 does not have the md5sum calculated during upload


4DNFIVMFPM8R does not have the md5sum calculated during upload


4DNFICAVAIV4 does not have the md5sum calculated during upload


4DNFIMNAPJXH does not have the md5sum calculated during upload


FINAL : 65 of 65 missing md5, 0 had no files on s3
FINAL : 65 of 0 started, 0 started (had complete run before)


In [15]:
### Check if md5 runs are in sync with file status
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')


md5_runs = '/search/?run_status=complete&status=in+review+by+lab&type=WorkflowRunAwsem&workflow.title=md5'
md5_hits = ff_utils.search_metadata(md5_runs , key=my_key)

print(len(md5_hits))

prob = 0
for a_run in md5_hits:
    f_status = a_run['input_files'][0]['value']['status']
    if f_status == 'uploaded':
        continue
    if f_status != a_run['status']:
        print(f_status, a_run['status'])
        prob += 1
    
print(prob)


patch = True
need_patch = 0
need_patch_run = 0
counter = 0

5101
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-relea

In [16]:
# patch the status
counter = 0
for a_run in md5_hits:
    counter += 1
    if counter % 100 == 0:
        print()
        print(counter)
    f_status = a_run['input_files'][0]['value']['status']
    if f_status in ['archived', 'released', 'released to project', 'pre-release']:
        print(f_status, a_run['status'])
        ff_utils.patch_metadata({"status": f_status}, obj_id=a_run['uuid'] ,key=my_key)
print(counter)
    
       


100

200

300

400

500

600

700

800

900

1000

1100

1200

1300
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre-release in review by lab
pre